In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [3]:
df=pd.read_csv("/kaggle/input/ship-fuel-consumption-and-co2-emissions-analysis/ship_fuel_efficiency.csv")

In [4]:
def explore_ship_data(df):
    print("Basic Dataset Information:")
    print(f"Number of rows: {df.shape[0]}")
    print(f"Number of columns: {df.shape[1]}")
    print("\nColumn names:")
    print(df.columns.tolist())

    print("Data Types and Non-Null Counts:")
    print(df.info())

    print("Summary Statistics:")
    print(df.describe().T)

    print("Null Values:")
    null_counts = df.isnull().sum()
    print(null_counts[null_counts > 0])
    
    return

In [5]:
explore_ship_data(df)

Basic Dataset Information:
Number of rows: 1440
Number of columns: 10

Column names:
['ship_id', 'ship_type', 'route_id', 'month', 'distance', 'fuel_type', 'fuel_consumption', 'CO2_emissions', 'weather_conditions', 'engine_efficiency']
Data Types and Non-Null Counts:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1440 entries, 0 to 1439
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ship_id             1440 non-null   object 
 1   ship_type           1440 non-null   object 
 2   route_id            1440 non-null   object 
 3   month               1440 non-null   object 
 4   distance            1440 non-null   float64
 5   fuel_type           1440 non-null   object 
 6   fuel_consumption    1440 non-null   float64
 7   CO2_emissions       1440 non-null   float64
 8   weather_conditions  1440 non-null   object 
 9   engine_efficiency   1440 non-null   float64
dtypes: float64(4), object(6)
memory 

In [6]:
def explore_Categorical(df):
    print("Unique Values in Categorical Columns:")
    categorical_columns = df.select_dtypes(include=['object']).columns
    for col in categorical_columns:
        print(f"\n{col}:")
        print(df[col].value_counts())
    return

In [7]:
explore_Categorical(df)

Unique Values in Categorical Columns:

ship_id:
ship_id
NG001    12
NG002    12
NG089    12
NG088    12
NG087    12
         ..
NG036    12
NG035    12
NG034    12
NG033    12
NG120    12
Name: count, Length: 120, dtype: int64

ship_type:
ship_type
Oil Service Boat    408
Tanker Ship         408
Surfer Boat         324
Fishing Trawler     300
Name: count, dtype: int64

route_id:
route_id
Port Harcourt-Lagos    389
Lagos-Apapa            388
Escravos-Lagos         369
Warri-Bonny            294
Name: count, dtype: int64

month:
month
January      120
February     120
March        120
April        120
May          120
June         120
July         120
August       120
September    120
October      120
November     120
December     120
Name: count, dtype: int64

fuel_type:
fuel_type
Diesel    899
HFO       541
Name: count, dtype: int64

weather_conditions:
weather_conditions
Calm        516
Stormy      462
Moderate    462
Name: count, dtype: int64


# Credit:

https://www.kaggle.com/code/jijagallery/fuel-consumption-predictor/notebook